In [11]:
import altair as alt
import pandas as pd
from vega_datasets import data as vega_data

In [2]:
chart = alt.Chart(data.cars.url).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N'
)
chart

alt.Chart(...)

In [3]:
chart.save('chart.html')

In [4]:
# TODO here figure out how to get our dataset integrated

In [5]:
# well, we can just pass in a dataframe
import altair as alt
import pandas as pd

data = pd.DataFrame({'x': ['A', 'B', 'C', 'D', 'E'],
                     'y': [5, 3, 6, 7, 2]})
chart2 = alt.Chart(data).mark_bar().encode(
    x='x',
    y='y',
)

In [6]:
# could now just do this with the data that we have...
games = pd.read_csv('../vgsales_clean.csv')
games.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [7]:
# nice!
# let's host this json on github and update...
data_url = 'https://raw.githubusercontent.com/6859-sp21/a4-video-game-sales/main/vgsales_clean.json'
chart_test = alt.Chart(data_url).mark_bar().encode(
    x='Year:O',
    y='sum(Global_Sales):Q'
)
chart_test

alt.Chart(...)

In [8]:
# let's do a stacked area chart of sales per year, colored by genre
chart_stacked = alt.Chart(data_url).mark_area().encode(
    x='Year:O',
    y='sum(Global_Sales):Q',
    color='Genre:N'
)
chart_stacked

alt.Chart(...)

In [12]:
cars = vega_data.cars.url

In [16]:
# selections!
brush = alt.selection_interval()  # selection of type "interval"
chart_select = alt.Chart(cars).mark_point().encode(
    x='Miles_per_Gallon:Q',
    y='Horsepower:Q',
    color=alt.condition(brush, 'Origin:N', alt.value('lightgray'))
).add_selection(
    brush
)
chart_select

alt.Chart(...)

In [17]:
chart_select.save('interactive_test.html')

In [18]:
# multiple charts!
chart = alt.Chart(cars).mark_point().encode(
    y='Horsepower:Q',
    color=alt.condition(brush, 'Origin:N', alt.value('lightgray'))
).properties(
    width=250,
    height=250
).add_selection(
    brush
)

chart.encode(x='Acceleration:Q') | chart.encode(x='Miles_per_Gallon:Q')

alt.HConcatChart(...)

In [19]:
# only care about x direction

chart = alt.Chart(cars).mark_point().encode(
    y='Horsepower:Q',
    color=alt.condition(brush, 'Origin:N', alt.value('lightgray'))
).properties(
    width=250,
    height=250
).add_selection(
    brush
)

chart.encode(x='Acceleration:Q') | chart.encode(x='Miles_per_Gallon:Q')


alt.HConcatChart(...)

In [22]:
def make_example(selector):
    cars = vega_data.cars.url

    return alt.Chart(cars).mark_rect().encode(
        x="Cylinders:O",
        y="Origin:N",
        color=alt.condition(selector, 'count()', alt.value('lightgray'))
    ).properties(
        width=300,
        height=180
    ).add_selection(
        selector
    )

In [23]:
interval = alt.selection_interval()
make_example(interval)

alt.Chart(...)

In [24]:
interval_x = alt.selection_interval(encodings=['x'], empty='none')
make_example(interval_x)

alt.Chart(...)

In [25]:
# bind to scales
scales = alt.selection_interval(bind='scales')

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N'
).add_selection(
    scales
)


alt.Chart(...)

In [26]:
single = alt.selection_single()
make_example(single)

alt.Chart(...)

In [28]:
# mouseover!
single_nearest = alt.selection_single(on='mouseover', nearest=True)
make_example(single_nearest)

alt.Chart(...)

In [29]:
# hold shift key to select multiple
multi = alt.selection_multi()
make_example(multi)

alt.Chart(...)

In [30]:
# paint the chart with mouseover
multi_mouseover = alt.selection_multi(on='mouseover', toggle=False, empty='none')
make_example(multi_mouseover)

alt.Chart(...)

In [32]:
# interesting combination of selections
alex = alt.selection_interval(
    on="[mousedown[event.altKey], mouseup] > mousemove",
    name='alex'
)
morgan = alt.selection_interval(
    on="[mousedown[event.shiftKey], mouseup] > mousemove",
    mark=alt.BrushConfig(fill="#fdbb84", fillOpacity=0.5, stroke="#e34a33"),
    name='morgan'
)

alt.Chart(cars).mark_rect().encode(
    x='Cylinders:O',
    y='Origin:O',
    color=alt.condition(alex | morgan, 'count()', alt.ColorValue("grey"))
).add_selection(
    alex, morgan
).properties(
    width=300,
    height=180
)


alt.Chart(...)

In [33]:
# select things in the legend!
selection = alt.selection_multi(fields=['Origin'])
color = alt.condition(selection,
                      alt.Color('Origin:N', legend=None),
                      alt.value('lightgray'))

scatter = alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=color,
    tooltip='Name:N'
)

legend = alt.Chart(cars).mark_point().encode(
    y=alt.Y('Origin:N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection
)

scatter | legend

alt.HConcatChart(...)

In [35]:
# altair supports dropdown elements! nice
input_dropdown = alt.binding_select(options=['Europe','Japan','USA'])
selection = alt.selection_single(fields=['Origin'], bind=input_dropdown, name='Country of ')
color = alt.condition(selection,
                    alt.Color('Origin:N', legend=None),
                    alt.value('lightgray'))

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color=color,
    tooltip='Name:N'
).add_selection(
    selection
)

alt.Chart(...)

In [36]:
# can also just filter directly
input_dropdown = alt.binding_select(options=['Europe','Japan','USA'])
selection = alt.selection_single(fields=['Origin'], bind=input_dropdown, name='Country of ')
color = alt.condition(selection,
                    alt.Color('Origin:N', legend=None),
                    alt.value('lightgray'))

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    tooltip='Name:N'
).add_selection(
    selection
).transform_filter(
    selection
)


alt.Chart(...)

In [38]:
# allow for zooming as well?
selection = alt.selection_interval(bind='scales')

alt.Chart(cars).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    tooltip='Name:N'
).add_selection(
    selection
)

alt.Chart(...)

In [40]:
# can access selector values in other things! (i.e. a condition)

import altair as alt
import pandas as pd
import numpy as np

rand = np.random.RandomState(42)

df = pd.DataFrame({
    'xval': range(100),
    'yval': rand.randn(100).cumsum()
})

slider = alt.binding_range(min=0, max=100, step=1, name='cutoff:')
selector = alt.selection_single(name="SelectorName", fields=['cutoff'],
                                bind=slider, init={'cutoff': 50})

alt.Chart(df).mark_point().encode(
    x='xval',
    y='yval',
    color=alt.condition(
        alt.datum.xval < selector.cutoff,
        alt.value('red'), alt.value('blue')
    )
).add_selection(
    selector
)

alt.Chart(...)